In [0]:
# 1. Widgets
dbutils.widgets.text("run_date", "2026-01-15")
v_date = dbutils.widgets.get("run_date")

from pyspark.sql.functions import col, to_timestamp, split, element_at

# 2. Read from Bronze
bronze_df = spark.table("ecommerce_bronze").filter(col("batch_date") == v_date)

# 3. Data Cleaning
silver_df = bronze_df.withColumn("event_time", to_timestamp(col("event_time"))) \
    .withColumn("main_category", split(col("category_code"), r"\.").getItem(0)) \
    .withColumn("sub_category", split(col("category_code"), r"\.").getItem(1)) \
    .dropDuplicates(["event_time", "user_id", "product_id"])

# 4. Write to Silver Table
silver_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_prod.silver.cleaned_events")

print(f"Silver layer complete for {v_date}")